In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
import FinancialDSL
using Dates

import FinancialDSL.Currencies.BRL
import FinancialDSL.Currencies.USD

import FinancialDSL.Core.WhenAt
import FinancialDSL.Core.Unit
import FinancialDSL.Core.Scale
import FinancialDSL.Core.Both
import FinancialDSL.Core.Amount
import FinancialDSL.Core.Either
import FinancialDSL.Core.Give
import FinancialDSL.Core.Worthless
import FinancialDSL.Core.SpotCurrency
import FinancialDSL.Core.DiscountFactor
import FinancialDSL.Core.Stock
import FinancialDSL.Core.Volatility

import FinancialDSL.Core.FixedIncomeEvent
import FinancialDSL.Core.FixedIncomeContract

import FinancialDSL.Core.StaticHedgingModel
import FinancialDSL.Core.FutureValueModel
import FinancialDSL.Core.BinomialModelDaily

import FinancialDSL.Core.FixedScenario

import FinancialDSL.Core.ContractAttributes
import FinancialDSL.Core.compile_pricer
import FinancialDSL.Core.compile_cashflow_pricer
import FinancialDSL.Core.price
import FinancialDSL.Core.eachcashflow

import FinancialDSL.Core.DeltaNormalExposuresMethod
import FinancialDSL.Core.DeltaGammaApproxExposuresMethod
import FinancialDSL.Core.exposures

import BusinessDays, InterestRates

In [ ]:
c1 = WhenAt(Date(2021, 1, 2), Amount(200USD))
c2 = WhenAt(Date(2030, 1, 2), Amount(800USD))
c = Both(c1, c2)

In [ ]:
c_exercise = Both(Amount(1USD), Give(Amount(4BRL)))
c_opt = WhenAt(Date(2021, 1, 2), Either(c_exercise, Worthless()))

# Forward FX Contact

$$ payoff = S_T - K $$

In [ ]:
k = 2.9 * 0.9 / 0.7 * BRL
s = 1USD
currency_fwd = WhenAt(Date(2019, 1, 2), Both( Amount(s), Give(Amount(k)) ))

* Textbook pricing formula

$$ f(S, PU_{cc}, PU_{PRE}) = S PU_{CC} - K PU_{PRE} $$

* Gradient

$$ \frac{\partial f}{\partial S} = PU_{cc} $$

$$ \frac{\partial f}{\partial PU_{CC}} = S $$

$$ \frac{\partial f}{\partial PU_{PRE}} = -K $$

Assuming normal distributions of returns for Risk Factors (delta-normal method)

If $K = 3.72857$ and current scenario is $ S = 2.9 $, $ PU_{PRE} = 0.7 $ and $ PU_{CC} = 0.9 $ , then:

$$ f = 0 $$

$$ EXP_S = S \frac{\partial f}{\partial S} = 2.61 $$

$$ EXP_{PU_{CC}} = PU_{CC} \frac{\partial f}{\partial PU_{CC}} = 2.61 $$

$$ EXP_{PU_{PRE}} = PU_{PRE} \frac{\partial f}{\partial PU_{PRE}} = -2.61 $$

In [ ]:
pricing_date = Date(2018, 5, 29)

# pricing model
currency_to_curves_map = Dict( "onshore" => Dict( :BRL => :PRE, :USD => :cpUSD ))
static_model = StaticHedgingModel(BRL, currency_to_curves_map)

# Scenario
scenario = FixedScenario()
scenario[SpotCurrency(USD)] = 2.9BRL # dolar price in BRL
scenario[DiscountFactor(:PRE, Date(2019, 1, 2))] = 0.7 # Discount for BRL curve
scenario[DiscountFactor(:cpUSD, Date(2019, 1, 2))] = 0.9 # Discount for USD curve

# contract attributes
attr = ContractAttributes(:riskfree_curves => "onshore", :carry_type => "none")

# pricer
pricer_currency_fwd = compile_pricer(pricing_date, static_model, currency_fwd, attr)

# executes the pricing function
price(pricer_currency_fwd, scenario)

In [ ]:
exposures_result = exposures(
    DeltaNormalExposuresMethod(),
    pricer_currency_fwd,
    scenario)

for (k, v) in exposures_result
    println("Exposure for $k = $v")
end

In [ ]:
pricer_currency_fwd

In [ ]:
@time pricer_interpreted = compile_pricer(pricing_date, static_model, currency_fwd, attr)

In [ ]:
@time pricer_native = compile_pricer(pricing_date, static_model, currency_fwd, attr, compiler=:native)

In [ ]:
price(pricer_interpreted, scenario)

In [ ]:
price(pricer_native, scenario)

In [ ]:
@time price(pricer_interpreted, scenario)

In [ ]:
@time price(pricer_native, scenario)

# Fixed Income Contracts

In [ ]:
c = FixedIncomeEvent(:AMORT, WhenAt(Date(2020, 2, 1), Amount(10.0BRL)))
c2 = FixedIncomeEvent(:INTEREST, WhenAt(Date(2019, 2, 1), Amount(5.0BRL)))
fi_contract = FixedIncomeContract([c, c2])

In [ ]:
pricing_date = Date(2018, 5, 29)
currency_to_curves_map = Dict( "onshore" => Dict( :BRL => :PRE, :USD => :cpUSD ))
static_model = StaticHedgingModel(BRL, currency_to_curves_map)
fv_model = FutureValueModel(static_model)
attr = ContractAttributes(:riskfree_curves => "onshore", :carry_type => "none")

empty_scenario = FixedScenario()
pricer = compile_cashflow_pricer(pricing_date, fv_model, fi_contract, attr)

for cf in eachcashflow(pricer, empty_scenario)
    println(cf)
end

# Binomial Model

In [ ]:
put = FinancialDSL.Core.american_put(:PETR4, 20.0BRL, Date(2020, 5, 19))

In [ ]:
pricing_date = Date(2020, 5, 14)
currency_to_curves_map = Dict(
    "onshore" => Dict( :BRL => :PRE, :USD => :cpUSD, :PETR4 => Symbol("PETR4 DIVIDEND YIELD") ))

static_model = StaticHedgingModel(BRL, currency_to_curves_map)

binomial_daily_model = BinomialModelDaily(
    static_model,
    Stock(:PETR4), InterestRates.BDays252(BusinessDays.BRSettlement()))

attr = ContractAttributes(:riskfree_curves => "onshore", :carry_type => "none")

scenario_fixed = FixedScenario()
scenario_fixed[Stock(:PETR4)] = 20.0BRL
scenario_fixed[DiscountFactor(:PRE, Date(2020, 5, 15))] = 0.9
scenario_fixed[DiscountFactor(:PRE, Date(2020, 5, 18))] = 0.8
scenario_fixed[DiscountFactor(:PRE, Date(2020, 5, 19))] = 0.7
scenario_fixed[DiscountFactor(Symbol("PETR4 DIVIDEND YIELD"), Date(2020, 5, 15))] = 0.95
scenario_fixed[DiscountFactor(Symbol("PETR4 DIVIDEND YIELD"), Date(2020, 5, 18))] = 0.93
scenario_fixed[DiscountFactor(Symbol("PETR4 DIVIDEND YIELD"), Date(2020, 5, 19))] = 0.91
scenario_fixed[Volatility(FinancialDSL.Core.Stock(:PETR4))] = 2.00 # 200%

put_pricer = compile_pricer(pricing_date, binomial_daily_model, put, attr)

# pricing
put_p = price(put_pricer, scenario_fixed)

In [ ]:
put_pricer

In [ ]:
native_put_pricer = compile_pricer(pricing_date, binomial_daily_model, put, attr, compiler=:native)
price(native_put_pricer, scenario_fixed)

In [ ]:
# exposures
put_exposures = exposures(DeltaGammaApproxExposuresMethod(), put_pricer, scenario_fixed)
for (k, v) in put_exposures
    println("Exposure for $k = $v")
end

In [ ]:
native_put_exposures = FinancialDSL.Core.exposures(FinancialDSL.Core.DeltaGammaApproxExposuresMethod(), native_put_pricer, scenario_fixed)

# Pricing ZCB with the Binomial Model

In [ ]:
zcb = WhenAt(Date(2020, 5, 19), Amount(1000.0BRL))

In [ ]:
pricing_date = Date(2020, 5, 14)

attr = ContractAttributes(:riskfree_curves => "onshore", :carry_type => "none")

pricer = compile_pricer(pricing_date, binomial_daily_model, zcb, attr)
p = price(pricer, scenario_fixed) # should result 1000 * 0.7

In [ ]:
zcb_exposures = exposures(DeltaGammaApproxExposuresMethod(), pricer, scenario_fixed)

In [ ]:
pricer